<a href="https://colab.research.google.com/github/longtran22/NMKHDL_nhom2_IT4930/blob/main/Resnet18_finetune_CRema_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub
import kagglehub
import os
# Download latest version
path = kagglehub.dataset_download("akaiinu/crema-d")

print("Path to dataset files:", path)
print("Số lượng file:", len(os.listdir(path+'/cremad/Image-01-FPS')))
import os, glob, pandas as pd

root_dir = path+"/cremad/Image-01-FPS"
rows = []
print(os.listdir(root_dir))
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    if not os.path.isdir(folder_path):
        continue

    # tách nhãn cảm xúc từ tên folder
    parts = folder.split("_")
    if len(parts) < 3:
        continue
    emotion = parts[2]  # ví dụ: ANG, HAP, SAD, NEU, FEA, DIS...

    # lấy tất cả ảnh trong folder đó
    for img_path in glob.glob(os.path.join(folder_path, "*.jpg")):
        rows.append([img_path, emotion])

df = pd.DataFrame(rows, columns=["image_path", "emotion"])
print("Số lượng ảnh:", len(df))
print("Các nhãn:", sorted(df["emotion"].unique()))
df.head()

emotions = sorted(df["emotion"].unique())
emo2idx = {emo: i for i, emo in enumerate(emotions)}
df["label"] = df["emotion"].map(emo2idx)
print(emo2idx)



from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class EmotionDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path, label = self.df.iloc[idx][["image_path", "label"]]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label





100%|██████████| 1.14G/1.14G [00:55<00:00, 22.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/akaiinu/crema-d/versions/1
Số lượng file: 7442
['1022_IWW_HAP_XX', '1033_WSI_ANG_XX', '1016_TSI_FEA_XX', '1019_TIE_FEA_XX', '1021_IEO_SAD_MD', '1073_IEO_DIS_MD', '1068_ITH_SAD_XX', '1023_WSI_SAD_XX', '1046_TAI_ANG_XX', '1012_IOM_DIS_XX', '1054_IOM_NEU_XX', '1082_IWW_DIS_XX', '1036_MTI_HAP_XX', '1016_TSI_NEU_XX', '1011_ITH_HAP_XX', '1071_MTI_DIS_XX', '1024_ITH_FEA_XX', '1003_TIE_SAD_XX', '1052_TIE_ANG_XX', '1059_ITH_ANG_XX', '1020_TIE_SAD_XX', '1085_IEO_FEA_MD', '1046_MTI_SAD_XX', '1082_ITH_FEA_XX', '1064_IEO_ANG_MD', '1071_TIE_ANG_XX', '1079_TAI_DIS_XX', '1046_IEO_HAP_LO', '1091_WSI_HAP_XX', '1072_ITH_NEU_XX', '1001_TSI_NEU_XX', '1079_IEO_DIS_MD', '1050_DFA_SAD_XX', '1027_TSI_SAD_XX', '1020_TSI_HAP_XX', '1024_IWW_ANG_XX', '1062_TIE_DIS_XX', '1006_WSI_HAP_XX', '1059_DFA_SAD_XX', '1077_IWL_ANG_XX', '1079_DFA_DIS_XX', '1016_IWW_NEU_XX', '1089_TIE_NEU_XX', '1076_TSI_ANG_XX', '1014_TAI_NEU_XX', '1012_IWL_HAP_XX', '1065_IWL_DIS_XX', '106

In [2]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=42)
train_loader = DataLoader(EmotionDataset(train_df, transform), batch_size=64, shuffle=True)
val_loader   = DataLoader(EmotionDataset(val_df, transform), batch_size=64)
!pip install torch tdqm


  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=f989ae682158082c070b72a780b249cf4ff284c4201b141b90e938e82c7b6dd7
  Stored in directory: /root/.cache/pip/wheels/af/02/71/aae0f7ee738abf19498353918ddae0f90a0d6ceb337b0bbc91
Successfully built tdqm


In [3]:
# =============================================================================
# PHẦN 1: CHUẨN BỊ DỮ LIỆU (DATA PREPARATION)
# =============================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import numpy as np

# 1.1. Sửa lỗi Data Leakage: Chia tập Train/Val dựa trên Actor ID
# Lấy ID diễn viên từ tên file (Ví dụ: 1006_TAI_NEU_XX -> ID là 1006)
# Giả định df đã có cột 'image_path' từ phần code trước của bạn
df['actor_id'] = df['image_path'].apply(lambda x: os.path.basename(x).split('_')[0])

unique_actors = df['actor_id'].unique()
train_actors, val_actors = train_test_split(unique_actors, test_size=0.2, random_state=42)

print(f"Tổng số diễn viên: {len(unique_actors)}")
print(f"Train actors: {len(train_actors)} | Val actors: {len(val_actors)}")

# Tạo DataFrame riêng biệt
train_df = df[df['actor_id'].isin(train_actors)].reset_index(drop=True)
val_df = df[df['actor_id'].isin(val_actors)].reset_index(drop=True)

print(f"Số ảnh Train: {len(train_df)} | Số ảnh Val: {len(val_df)}")

# 1.2. Định nghĩa Transforms (Tăng cường dữ liệu mạnh mẽ cho Train)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Resize về chuẩn ResNet
    transforms.RandomHorizontalFlip(p=0.5), # Lật ảnh ngẫu nhiên
    transforms.RandomRotation(10),       # Xoay nhẹ +/- 10 độ
    transforms.ColorJitter(brightness=0.2, contrast=0.2), # Chỉnh sáng/tương phản
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Chỉ resize chuẩn
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 1.3. Khởi tạo Dataset và DataLoader
# (Sử dụng class EmotionDataset bạn đã định nghĩa ở cell trước)
train_dataset = EmotionDataset(train_df, transform=train_transform)
val_dataset = EmotionDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

# =============================================================================
# PHẦN 2: XÂY DỰNG MODEL RESNET18 (TRANSFER LEARNING)
# =============================================================================

def build_resnet18_finetune_layer4(num_classes):
    # 1. Load Pre-trained weights
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

    # 2. ĐÓNG BĂNG TOÀN BỘ TRƯỚC (Freeze All)
    for param in model.parameters():
        param.requires_grad = False

    # 3. MỞ BĂNG LAYER 4 (Unfreeze Layer 4)
    # Đây là khối Conv cuối cùng, chứa các đặc trưng cao cấp nhất
    for param in model.layer4.parameters():
        param.requires_grad = True

    # 4. THAY THẾ CLASSIFICATION HEAD
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(256, num_classes)
    )

    return model

# Khởi tạo model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Số class dựa trên dữ liệu của bạn (thường là 6 hoặc 7)
num_classes = len(unique_actors) if 'unique_actors' not in locals() else 6
# Lưu ý: Sửa lại số 6 cứng nếu bạn biết chắc chắn số emotion, code trên lấy ví dụ an toàn
num_classes = 6

print(f"Đang khởi tạo model Fine-tune Layer 4 trên: {device}")
model = build_resnet18_finetune_layer4(num_classes).to(device)

# =============================================================================
# PHẦN 2: THIẾT LẬP OPTIMIZER VỚI 2 TỐC ĐỘ HỌC (QUAN TRỌNG)
# =============================================================================

# Chúng ta chia tham số thành 2 nhóm để áp dụng Learning Rate khác nhau
optimizer = optim.Adam([
    {
        # Nhóm 1: Layer 4 (Backbone) -> Học rất chậm để bảo toàn kiến thức cũ
        'params': model.layer4.parameters(),
        'lr': 1e-5  # 0.00001
    },
    {
        # Nhóm 2: FC Layers (Head) -> Học nhanh hơn vì mới khởi tạo ngẫu nhiên
        'params': model.fc.parameters(),
        'lr': 1e-3  # 0.001
    }
])

criterion = nn.CrossEntropyLoss()

# =============================================================================
# PHẦN 3: TRAINING LOOP
# =============================================================================

epochs = 50 # Fine-tuning cần nhiều epoch hơn một chút để Layer 4 nhích dần
best_val_loss = float('inf')
best_model_path = 'best_resnet18_finetune4.pth'

# Scheduler: Giảm LR nếu loss đi ngang (Giúp hội tụ tốt hơn ở giai đoạn cuối)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

for epoch in range(epochs):
    # --- TRAIN ---
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)

    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        loop.set_postfix(loss=loss.item())

    avg_train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    # --- VALIDATION ---
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val

    # Cập nhật Scheduler
    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch+1}: "
          f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # --- CHECKPOINT ---
    if avg_val_loss < best_val_loss:
        print(f"🚀 Val Loss giảm từ {best_val_loss:.4f} xuống {avg_val_loss:.4f}. Saving model...")
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)

print("="*30)
print("Hoàn tất Fine-tuning!")
model.load_state_dict(torch.load(best_model_path))

Tổng số diễn viên: 64
Train actors: 51 | Val actors: 13
Số ảnh Train: 15842 | Số ảnh Val: 7457
Đang khởi tạo model Fine-tune Layer 4 trên: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 129MB/s]


Epoch 1/50 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

Epoch 1: Train Loss: 1.7752 | Train Acc: 21.59% | Val Loss: 1.6913 | Val Acc: 29.31%
🚀 Val Loss giảm từ inf xuống 1.6913. Saving model...


Epoch 2/50 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

...

Epoch 49/50 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

Epoch 49: Train Loss: 1.1417 | Train Acc: 54.00% | Val Loss: 0.9406 | Val Acc: 65.27%
🚀 Val Loss giảm từ 0.9448 xuống 0.9406. Saving model...


Epoch 50/50 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

Epoch 50: Train Loss: 1.1360 | Train Acc: 54.00% | Val Loss: 0.9494 | Val Acc: 65.46%
Hoàn tất Fine-tuning!


<All keys matched successfully>

In [ ]:
# =============================================================================
# TRAIN TIẾP (RESUME TRAINING) - 20 EPOCH NỮA
# =============================================================================

# 1. Cấu hình
epochs = 20  # Train thêm 20 epoch
best_model_path = 'best_resnet18_finetune4.pth'

# 2. Load lại model tốt nhất từ lần chạy trước
print(f"🔄 Đang load lại model tốt nhất từ: {best_model_path}")
# Đảm bảo model đã được khởi tạo (biến 'model' từ cell trước)
# Nếu bạn đã lỡ xóa biến model, hãy chạy lại hàm build_resnet18_finetune_layer4 trước
model.load_state_dict(torch.load(best_model_path))
model.to(device)

# 3. Tính toán Loss hiện tại để làm mốc chuẩn (Baseline)
# Để tránh việc lưu đè model kém hơn, ta cần biết model hiện tại tốt mức nào
print("📊 Đang kiểm tra hiệu năng hiện tại của model...")
model.eval()
current_val_loss = 0.0
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        current_val_loss += loss.item()
best_val_loss = current_val_loss / len(val_loader)
print(f"   => Mức chuẩn Val Loss hiện tại: {best_val_loss:.4f}")

# 4. Thiết lập lại Optimizer & Scheduler
# Khi train tiếp, ta vẫn giữ nguyên learning rate như cũ hoặc giảm nhẹ tùy ý
# Ở đây tôi giữ nguyên config như cell trước để đúng ý bạn
optimizer = optim.Adam([
    {'params': model.layer4.parameters(), 'lr': 1e-6},
    {'params': model.fc.parameters(),     'lr': 1e-4}
])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

print(f"\n🚀 BẮT ĐẦU TRAIN TIẾP {epochs} EPOCH...")
print("="*60)

for epoch in range(epochs):
    # --- TRAIN ---
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)

    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        loop.set_postfix(loss=loss.item())

    avg_train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    # --- VALIDATION ---
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val

    # Cập nhật Scheduler
    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch+1}: "
          f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # --- CHECKPOINT ---
    # Chỉ lưu nếu Loss thấp hơn kỷ lục cũ (best_val_loss)
    if avg_val_loss < best_val_loss:
        print(f"🚀 Val Loss giảm từ {best_val_loss:.4f} xuống {avg_val_loss:.4f}. Saving model...")
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)

print("="*30)
print("Hoàn tất Training bổ sung!")
# Load lại model tốt nhất cuối cùng
model.load_state_dict(torch.load(best_model_path))

🔄 Đang load lại model tốt nhất từ: best_resnet18_finetune4.pth
📊 Đang kiểm tra hiệu năng hiện tại của model...
   => Mức chuẩn Val Loss hiện tại: 0.8652

🚀 BẮT ĐẦU TRAIN TIẾP 20 EPOCH...


Epoch 1/20 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

Epoch 1: Train Loss: 1.0229 | Train Acc: 59.17% | Val Loss: 0.8874 | Val Acc: 68.75%


Epoch 2/20 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

Epoch 2: Train Loss: 1.0133 | Train Acc: 59.25% | Val Loss: 0.8835 | Val Acc: 69.14%


...

Epoch 6/20 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

Epoch 6: Train Loss: 1.0114 | Train Acc: 59.56% | Val Loss: 0.8819 | Val Acc: 69.21%


Epoch 7/20 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]

Epoch 7: Train Loss: 1.0230 | Train Acc: 58.94% | Val Loss: 0.8731 | Val Acc: 69.04%


Epoch 8/20 [Train]:   0%|          | 0/248 [00:00<?, ?it/s]